<a href="https://colab.research.google.com/github/jhc154/tensorflow_tutorial_complement/blob/master/tf_object_detect_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on Google's  [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). This notebook combines other sources and my own troubleshooting to see whether the custom image detection worked. Key modifications to run object detector based on http://www.insightsbot.com/tensorflow-object-detection-tutorial-on-images/. BUILT TO RUN FROM GOOGLE COLAB.

In [0]:
!pip install -U --pre tensorflow=="2.*"

In [0]:
!pip install pycocotools

In [0]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [0]:
%%bash 
cd models/research
pip install .

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
from os import listdir
from os.path import isfile, join

%matplotlib inline

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
    # Path to frozen detection graph. This is the actual model that is used for the object detection.
    PATH_TO_CKPT = '/content/drive/My Drive/TensorFlow/workspace/training_demo/pre-trained-model/faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb'
    
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

In [0]:
    # List of the strings that is used to add correct label for each box.
    #Set PATH_TO_LABELS to the .pbtxt that you created in the tutorial if you are using your custom model
    #If using the pre-trained model, most likely use the mscoco_label_map.pbtxt that comes in the TensorFlow/models/research/object_detection folder

    PATH_TO_LABELS =  '/content/drive/My Drive/TensorFlow/models/research/object_detection/data/mscoco_label_map.pbtxt'
    
    NUM_CLASSES=90 #set to 90 if using mscoco; otherwise set to 1 when training the model on a single label
    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)

In [0]:
    #no changes are necessary to this cell, just run it to create the function
    def load_image_into_numpy_array(image):
        (im_width, im_height) = image.size
        return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [0]:
    #no changes are necessary to this cell, just run it to create the function
    def run_inference_for_single_image(image, graph):
        with graph.as_default():
            with tf.Session() as sess:
                # Get handles to input and output tensors
                ops = tf.get_default_graph().get_operations()
                all_tensor_names = {output.name for op in ops for output in op.outputs}
                tensor_dict = {}
                for key in ['num_detections', 'detection_boxes', 'detection_scores','detection_classes', 'detection_masks']:
                    tensor_name = key + ':0'
                    if tensor_name in all_tensor_names:
                        tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
                    #END if tensor_name in
                    if 'detection_masks' in tensor_dict:
                        # The following processing is only for single image
                        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                            detection_masks, detection_boxes, image.shape[0], image.shape[1])
                        detection_masks_reframed = tf.cast(
                            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                        # Follow the convention by adding back the batch dimension
                        tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
                    #END IF DETECTION MASKS
                
                #END FOR KEY LOOP
                    
                image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
                # Run inference
                output_dict = sess.run(tensor_dict,
                                     feed_dict={image_tensor: np.expand_dims(image, 0)})
                # all outputs are float32 numpy arrays, so convert types as appropriate
                output_dict['num_detections'] = int(output_dict['num_detections'][0])
                output_dict['detection_classes'] = output_dict[
                  'detection_classes'][0].astype(np.uint8)
                output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
                output_dict['detection_scores'] = output_dict['detection_scores'][0]
                if 'detection_masks' in output_dict:
                    output_dict['detection_masks'] = output_dict['detection_masks'][0]
        return output_dict

In [0]:
    #no changes are necessary to this cell, just run it to create the function
    def Run_Object_Detection_On_Images(images_path):
        IMAGE_SIZE = (12, 8)
        for image_path in images_path:
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            output_dict = run_inference_for_single_image(image_np, detection_graph)
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              output_dict['detection_boxes'],
              output_dict['detection_classes'],
              output_dict['detection_scores'],
              category_index,
              instance_masks=output_dict.get('detection_masks'),
              use_normalized_coordinates=True,
              line_thickness=5)
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)

In [0]:
#set path for TEST_IMAGES_BASE_PATH to whatever folder has images to test object detectors on

#I chose about 10 images from my dataset and placed them in a /dev folder
#Try running this object detector with a few different models before training to see how they perform
#Run this object detector using your custom model after training to compare it against the baseline models

TEST_IMAGES_BASE_PATH = "/content/drive/My Drive/TensorFlow/workspace/training_demo/images/dev/"
TEST_IMAGES_PATHS = [TEST_IMAGES_BASE_PATH+f for f in listdir(TEST_IMAGES_BASE_PATH) if isfile(join(TEST_IMAGES_BASE_PATH, f))]

In [0]:
Run_Object_Detection_On_Images(TEST_IMAGES_PATHS)

